In [1]:
from sage.symbolic.integration.integral import integrate
# Anillo de polinomios de dos variables
R.<x,y> = PolynomialRing(QQ, order='degrevlex')

# Producto escalar y funcional de Hermite (en dos variables)
dot_H = lambda f, g: (f*g*e**(-x**2-y**2)).integrate(x, -infinity, infinity).integrate(y, -infinity, infinity)
L_H = lambda P : dot_H(P,1)

# Parámetros para las dos familias de polinomios de Laguerre
alpha = 1
beta = 3
# Producto escalar y funcional de Laguerre (en dos variables)
dot_L = lambda f, g: (f*g*x**(alpha)*y**(beta)*e**(-x-y)).integrate(x, 0, infinity).integrate(y, 0, infinity)
L_L = lambda P : dot_L(P,1)


Como la función nativa de sage para el cálculo de los polinomios de Laguerre es poco intuitiva, creamos nuestra propia función que utiliza la fórmula de Rodrigues para calcular polinomios de Laguerre

In [2]:
def laguerre(n, alpha, var):
    ''' Calcula el polinomio de Laguerre de grado n y parametro 
        'alpha' utilizando la formula de Rodrigues
    '''
    Bn = 1/factorial(n)
    rho_inv = pow(var,-alpha)*e**var
    der = derivative(var**(n+alpha)*e**(-var),var,n)
    return (Bn*rho_inv*der).full_simplify()

Recordamos la notación
$$
\mathbb P_n = G_{n,n}\mathbb X_n + G_{n,n-1} \mathbb X_{n-1} +\cdots G_{n,1}\mathbb X_1 + G_{n,0}\mathbb X_0,
$$
donde $\mathbb X_j = (x^j, x^{j-1}y,\dots,xy^{j-1}, y^j)^T$ es un vector de $j+1$ filas, por lo que $\mathbb P_n$ es un vector columna de $n+1$ componentes (polinomios) y cada matrix $G_{n,j}$ es una matriz $(n+1)\times(j+1)$.

Crearemos código que, a partir de los polinomios de Hermite y Laguerre de una variable, calcule los polinomios $\mathbb P_n$ de Hermite y de Laguerre, para posteriormente buscar ortogonalidad múltiple.

Previamente, dado un vector columna de polinomios de grado $n$, $\mathbb P_n$, conviene encontrar matrices $G_{n,n-1},\dots,G_{n,0}$ que representen su representación de acuerdo a $\mathbb P_n = \displaystyle\sum_{j=0}^n G_{n,j}\mathbb X_j$.

Pensemos en un polinomio aleatorio, por ejemplo:

In [3]:
p = 8*x^2*y - 4*y

Busquemos alguna forma de extraer sus coeficientes. Para ello, comencemos por encontrar sus coeficientes de un grado fijo.

In [4]:
coeficientes_grado = lambda p,n : [p.coefficient({x:n-j, y:j}) for j in range(n+1)]
coeficientes_grado(p,3),coeficientes_grado(p,2), coeficientes_grado(p,1), coeficientes_grado(p,0)

([0, 8, 0, 0], [0, 0, 0], [0, -4], [0])

Y agrupamos las distintas listas de coeficientes en matrices fila.

In [5]:
coeficientes = lambda p : [Matrix(coeficientes_grado(p,j)) for j in range(p.degree()+1)]
coeficientes(p)

[[0], [ 0 -4], [0 0 0], [0 8 0 0]]

Creamos los vectores $\mathbb X_j$:

In [29]:
monomios_grado = lambda n : [R(x**(n-j) * y**j) for j in range(n+1)]
X = [transpose(matrix(monomios_grado(j))) for j in range(p.degree()+1)]
X

[
                 [  x^3]
          [x^2]  [x^2*y]
     [x]  [x*y]  [x*y^2]
[1], [y], [y^2], [  y^3]
]

Comprobemos que, si $g$ es el vector `coeficientes(p)` $\displaystyle\sum_{j=0}^3 g_j \mathbb X_j = p$.

In [7]:
matrices_coefs = coeficientes(p)
sum(matrices_coefs[j]*X[j] for j in range(p.degree()+1))[0,0] == p

True

Por lo que con el uso reiterado de la función `coeficientes` podemos obtener las matrices $G_{n,j}$. Véamoslo mejor en la práctica:

In [8]:
def coeficientes(p):
    p = R(p)
    coeficientes_grado = lambda p,n : [p.coefficient({x:n-j, y:j}) for j in range(n+1)]
    return [coeficientes_grado(p,j) for j in range(p.degree()+1)]

# Ejemplo de biortogonalidad Hermite-Laguerre
## Polinomios producto de Hermite

La siguiente función calcula los polinomios de Hermite $P_{n-k,k}=H_{n-k}(x) H_k(y)$ y los coloca en un vector columna $\mathbb P_n$.

In [9]:
def P_Hermite(n):
    P = [R(hermite(n-k,x)*hermite(k,y)) for k in range(n+1)]
    P = [p/p.lc() for p in P] # polinomios monicos
    return transpose(matrix(P))

In [10]:
P_Hermite(2)

[x^2 - 1/2]
[      x*y]
[y^2 - 1/2]

Obsérvese el monomio líder de cada polinomio. Estos vectores columna representan los vectores columna $\mathbb P_n$. Veamos ahora cómo obtener su representación $\mathbb P_n = \mathbb X_n + G_{n,n-1} \mathbb X_{n-1} +\cdots G_{n,1}\mathbb X_1 + G_{n,0}\mathbb X_0$. Elegimos un $n$ al azar, por ejemplo $n=3$.

In [11]:
P = P_Hermite(3)
P

[  x^3 - 3/2*x]
[x^2*y - 1/2*y]
[x*y^2 - 1/2*x]
[  y^3 - 3/2*y]

Extraemos los coeficientes de cada polinomio y los colocamos en matrices

In [12]:
coefs_P = [coeficientes(p) for p in P.list()]
matrices_G = [Matrix([coefs_P[j][i] for j in range(len(coefs_P))]) for i in range(4)]
matrices_G

[
[0]  [-3/2    0]  [0 0 0]  [1 0 0 0]
[0]  [   0 -1/2]  [0 0 0]  [0 1 0 0]
[0]  [-1/2    0]  [0 0 0]  [0 0 1 0]
[0], [   0 -3/2], [0 0 0], [0 0 0 1]
]

Comprobamos

In [13]:
sum(matriz*monomio for matriz, monomio in zip(matrices_G, X)) - P # Debe ser 0

[0]
[0]
[0]
[0]

Tenemos por tanto una forma de extraer las matrices $G_{n,j}$ de cada polinomio. Ahora, extenderemos el producto escalar estándar de Hermite para poder hacer productos escalares $\left\langle \mathbb P_i,\mathbb P_j \right\rangle$. La idea es, teniendo en cuenta que $\mathbb P_i\cdot\mathbb P_j^T$ es una matriz de dimensión $(i+1)\times(j+1)$, la ortogonalidad se manifiesta si, al aplicar el funcional de Hermite $\mathcal L$ a cada entrada de la matriz el resultado es una matriz nula si $i\neq j$ y una matriz cuadrada no nula si $i=j$: 

In [14]:
def dotP_H(P, Q):
    M = P * transpose(Q)
    return matrix([[L_H(M[i,j]) for i in range(P.nrows())] for j in range(Q.nrows())])

In [15]:
dotP_H(P_Hermite(3), P_Hermite(4))

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

In [16]:
dotP_H(P_Hermite(3), P_Hermite(3))

[3/4*pi      0      0      0]
[     0 1/4*pi      0      0]
[     0      0 1/4*pi      0]
[     0      0      0 3/4*pi]

Consideremos el sistema de polinomios ortogonales de Hermite y comprobemos la ortogonalidad entre grados utilizando esta funcionalidad:

In [17]:
SPO_Hermite = [P_Hermite(j) for j in range(6)]
SPO_Hermite

[
                       [  x^3 - 3/2*x]
          [x^2 - 1/2]  [x^2*y - 1/2*y]
     [x]  [      x*y]  [x*y^2 - 1/2*x]
[1], [y], [y^2 - 1/2], [  y^3 - 3/2*y],

[                x^4 - 3*x^2 + 3/4]
[                  x^3*y - 3/2*x*y]
[x^2*y^2 - 1/2*x^2 - 1/2*y^2 + 1/4]
[                  x*y^3 - 3/2*x*y]
[                y^4 - 3*y^2 + 3/4],

[                 x^5 - 5*x^3 + 15/4*x]
[              x^4*y - 3*x^2*y + 3/4*y]
[x^3*y^2 - 1/2*x^3 - 3/2*x*y^2 + 3/4*x]
[x^2*y^3 - 3/2*x^2*y - 1/2*y^3 + 3/4*y]
[              x*y^4 - 3*x*y^2 + 3/4*x]
[                 y^5 - 5*y^3 + 15/4*y]
]

In [18]:
for i in range(len(SPO_Hermite)):
    P = SPO_Hermite[i]
    for j in range(i+1):
        Q = SPO_Hermite[j]
        print("<P_" + str(i) + ", P_" + str(j) + "> = " + str(dotP_H(P,Q)))
        
    

<P_0, P_0> = [pi]
<P_1, P_0> = [0 0]
<P_1, P_1> = [1/2*pi      0]
[     0 1/2*pi]
<P_2, P_0> = [0 0 0]
<P_2, P_1> = [0 0 0]
[0 0 0]
<P_2, P_2> = [1/2*pi      0      0]
[     0 1/4*pi      0]
[     0      0 1/2*pi]
<P_3, P_0> = [0 0 0 0]
<P_3, P_1> = [0 0 0 0]
[0 0 0 0]
<P_3, P_2> = [0 0 0 0]
[0 0 0 0]
[0 0 0 0]
<P_3, P_3> = [3/4*pi      0      0      0]
[     0 1/4*pi      0      0]
[     0      0 1/4*pi      0]
[     0      0      0 3/4*pi]
<P_4, P_0> = [0 0 0 0 0]
<P_4, P_1> = [0 0 0 0 0]
[0 0 0 0 0]
<P_4, P_2> = [0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
<P_4, P_3> = [0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
<P_4, P_4> = [3/2*pi      0      0      0      0]
[     0 3/8*pi      0      0      0]
[     0      0 1/4*pi      0      0]
[     0      0      0 3/8*pi      0]
[     0      0      0      0 3/2*pi]
<P_5, P_0> = [0 0 0 0 0 0]
<P_5, P_1> = [0 0 0 0 0 0]
[0 0 0 0 0 0]
<P_5, P_2> = [0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
<P_5, P_3> = [0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0

## Polinomios producto de Laguerre

El procedimiento es análogo al seguido con los polinomios de Hermite. Creamos una función que calcula $P_{n-k,k}(x,y)=L^\alpha_{n-k}(x)L^\beta_k(y)$ y los coloca en un vector columna de tamaño $n+1$, que será nuestro $\mathbb P_n$.

In [19]:
def P_Laguerre(n):
    P = [R(laguerre(n-k,alpha,x)*laguerre(k,beta,y)) for k in range(n+1)]
    P = [p/p.lc() for p in P] # polinomios monicos
    return transpose(matrix(P))

In [20]:
P_Laguerre(2)

[      x^2 - 6*x + 6]
[x*y - 4*x - 2*y + 8]
[    y^2 - 10*y + 20]

Generalizamos el producto escalar para que sea aplicado a parejas de vectores de polinomios.

In [21]:
def dotP_L(P, Q):
    M = P * transpose(Q)
    return matrix([[L_L(M[i,j]) for i in range(P.nrows())] for j in range(Q.nrows())])

De nuevo, comprobamos que, cuando los grados difieren, el resultado de aplicar el producto escalar es una matriz nula, pero si hacemos el producto de un vector de grado $n$ consigo mismo, el resultado es una matriz cuadrada no nula.

In [22]:
dotP_L(P_Laguerre(3), P_Laguerre(4))

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

In [23]:
dotP_L(P_Laguerre(3), P_Laguerre(3))

[ 864    0    0    0]
[   0  288    0    0]
[   0    0  480    0]
[   0    0    0 4320]

Calculamos algunos de estos polinomios y verificamos su ortogonalidad

In [24]:
SPO_Laguerre = [P_Laguerre(j) for j in range(4)]
SPO_Laguerre

[
              [      x^2 - 6*x + 6]
     [x - 2]  [x*y - 4*x - 2*y + 8]
[1], [y - 4], [    y^2 - 10*y + 20],

[                 x^3 - 12*x^2 + 36*x - 24]
[  x^2*y - 4*x^2 - 6*x*y + 24*x + 6*y - 24]
[x*y^2 - 10*x*y - 2*y^2 + 20*x + 20*y - 40]
[                y^3 - 18*y^2 + 90*y - 120]
]

In [25]:
for i in range(len(SPO_Laguerre)):
    P = SPO_Laguerre[i]
    for j in range(i+1):
        Q = SPO_Laguerre[j]
        print("<P_" + str(i) + ", P_" + str(j) + "> = " + str(dotP_L(P,Q)))
        
    

<P_0, P_0> = [6]
<P_1, P_0> = [0 0]
<P_1, P_1> = [12  0]
[ 0 24]
<P_2, P_0> = [0 0 0]
<P_2, P_1> = [0 0 0]
[0 0 0]
<P_2, P_2> = [ 72   0   0]
[  0  48   0]
[  0   0 240]
<P_3, P_0> = [0 0 0 0]
<P_3, P_1> = [0 0 0 0]
[0 0 0 0]
<P_3, P_2> = [0 0 0 0]
[0 0 0 0]
[0 0 0 0]
<P_3, P_3> = [ 864    0    0    0]
[   0  288    0    0]
[   0    0  480    0]
[   0    0    0 4320]


## Mezclando ambas medidas

In [26]:
def a(i):
    return SR.var('a{}'.format(i))
incognitas = [a(i) for i in range(1,25)]
G32 = matrix([
    [ a(1),  a(2),  a(3)],
    [ a(4),  a(5),  a(6)],
    [ a(7),  a(8),  a(9)],
    [a(10), a(11), a(12)]
])

G31 = matrix([
    [a(13), a(14)],
    [a(15), a(16)],
    [a(17), a(18)],
    [a(19), a(20)]
])

G30 = matrix([
    [a(21)],[a(22)],[a(23)],[a(24)]
])

In [31]:
P3 = X[3] + G32*X[2] + G31*X[1] + G30*X[0]
P3 = matrix(P3)
P3

[   a1*x^2 + x^3 + a2*x*y + a3*y^2 + a13*x + a14*y + a21]
[ a4*x^2 + a5*x*y + x^2*y + a6*y^2 + a15*x + a16*y + a22]
[ a7*x^2 + a8*x*y + a9*y^2 + x*y^2 + a17*x + a18*y + a23]
[a10*x^2 + a11*x*y + a12*y^2 + y^3 + a19*x + a20*y + a24]

In [32]:
ecs = dotP_H(P3, X[1]).list() + dotP_H(P3, X[0]).list() + dotP_L(P3, X[1]).list() + dotP_L(P3, X[0]).list()
ecs

[1/4*sqrt(pi)*(2*sqrt(pi)*a13 + 3*sqrt(pi)),
 1/2*pi*a15,
 1/4*sqrt(pi)*(2*sqrt(pi)*a17 + sqrt(pi)),
 1/2*pi*a19,
 1/2*pi*a14,
 1/4*sqrt(pi)*(2*sqrt(pi)*a16 + sqrt(pi)),
 1/2*pi*a18,
 3/4*pi + 1/2*pi*a20,
 1/2*pi*a3 + 1/2*sqrt(pi)*(sqrt(pi)*a1 + 2*sqrt(pi)*a21),
 1/2*pi*a6 + 1/2*sqrt(pi)*(2*sqrt(pi)*a22 + sqrt(pi)*a4),
 1/2*pi*a9 + 1/2*sqrt(pi)*(2*sqrt(pi)*a23 + sqrt(pi)*a7),
 1/2*pi*a12 + 1/2*sqrt(pi)*(sqrt(pi)*a10 + 2*sqrt(pi)*a24),
 144*a1 + 36*a13 + 48*a14 + 144*a2 + 12*a21 + 240*a3 + 720,
 36*a15 + 48*a16 + 12*a22 + 144*a4 + 144*a5 + 240*a6 + 576,
 36*a17 + 48*a18 + 12*a23 + 144*a7 + 144*a8 + 240*a9 + 720,
 144*a10 + 144*a11 + 240*a12 + 36*a19 + 48*a20 + 12*a24 + 1440,
 144*a1 + 48*a13 + 120*a14 + 240*a2 + 24*a21 + 720*a3 + 576,
 48*a15 + 120*a16 + 24*a22 + 144*a4 + 240*a5 + 720*a6 + 720,
 48*a17 + 120*a18 + 24*a23 + 144*a7 + 240*a8 + 720*a9 + 1440,
 144*a10 + 240*a11 + 720*a12 + 48*a19 + 120*a20 + 24*a24 + 5040,
 36*a1 + 12*a13 + 24*a14 + 48*a2 + 6*a21 + 120*a3 + 144,
 12*a15 + 2

In [33]:
len(ecs)

24

In [34]:
sol = solve(ecs, incognitas)[0]
sol

[a1 == (-1149/52),
 a2 == (1275/52),
 a3 == (-255/52),
 a4 == (-41/2),
 a5 == (99/4),
 a6 == (-11/2),
 a7 == (-123/4),
 a8 == (165/4),
 a9 == (-41/4),
 a10 == (-2343/26),
 a11 == (7029/52),
 a12 == (-933/26),
 a13 == (-3/2),
 a14 == 0,
 a15 == 0,
 a16 == (-1/2),
 a17 == (-1/2),
 a18 == 0,
 a19 == 0,
 a20 == (-3/2),
 a21 == (27/2),
 a22 == 13,
 a23 == (41/2),
 a24 == 63]

In [35]:
coefs_pol = [s.right() for s in sol]

In [36]:
G32 = matrix([
    coefs_pol[0:3],
    coefs_pol[3:6],
    coefs_pol[6:9],
    coefs_pol[9:12],
])

G31 = matrix([
    coefs_pol[12:14],
    coefs_pol[14:16],
    coefs_pol[16:18],
    coefs_pol[18:20]
])

G30 = matrix([
    coefs_pol[20:21],
    coefs_pol[21:22],
    coefs_pol[22:23],
    coefs_pol[23:24]
])

In [37]:
P3 = X[3] + G32*X[2] + G31*X[1] + G30*X[0]
P3 = matrix(P3)
P3

[x^3 - 1149/52*x^2 + 1275/52*x*y - 255/52*y^2 - 3/2*x + 27/2]
[        x^2*y - 41/2*x^2 + 99/4*x*y - 11/2*y^2 - 1/2*y + 13]
[    x*y^2 - 123/4*x^2 + 165/4*x*y - 41/4*y^2 - 1/2*x + 41/2]
[  y^3 - 2343/26*x^2 + 7029/52*x*y - 933/26*y^2 - 3/2*y + 63]

In [38]:
dotP_H(P3,X[1]), dotP_H(P3,X[0]), dotP_L(P3,X[1]), dotP_L(P3,X[0])   # (2,2)

(
[0 0 0 0]             [0 0 0 0]           
[0 0 0 0], [0 0 0 0], [0 0 0 0], [0 0 0 0]
)

In [39]:
P3

[x^3 - 1149/52*x^2 + 1275/52*x*y - 255/52*y^2 - 3/2*x + 27/2]
[        x^2*y - 41/2*x^2 + 99/4*x*y - 11/2*y^2 - 1/2*y + 13]
[    x*y^2 - 123/4*x^2 + 165/4*x*y - 41/4*y^2 - 1/2*x + 41/2]
[  y^3 - 2343/26*x^2 + 7029/52*x*y - 933/26*y^2 - 3/2*y + 63]

In [47]:
p = P3[3][0]
p.factor()

y^3 - 2343/26*x^2 + 7029/52*x*y - 933/26*y^2 - 3/2*y + 63

Ninguno factoriza

In [44]:
q = R(x^2-y^2)
q.factor()

(x - y) * (x + y)